# Homework 1| Danny Stein



In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings

from sklearn.linear_model import LinearRegression
import scipy.optimize as optimize

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13
import sys
sys.path.insert(0,'C:/Users/dcste/OneDrive/fixed_income/Fixed_Income/treasury_cmds.py')
from treasury_cmds import *

In [2]:
df = pd.read_excel('C:/Users/dcste/OneDrive/fixed_income/Fixed_Income/treasury_quotes_2022-09-30.xlsx')
df.columns = df.columns.str.upper()
df.sort_values('TMATDT', inplace = True)
df.set_index('KYTREASNO', inplace = True)
df.head()

,KYCRSPID_X,CALDT,TDBID,TDASK,TDNOMPRC,TDNOMPRC_FLG,TDSOURCR,TDACCINT,TDRETNUA,TDYLD,...,TFCALDT,TNOTICE,IYMCN,ITYPE,IUNIQ,ITAX,IFLWR,TBANKDT,TSTRIPELIG,TFRGNTGT
KYTREASNO,,,,,,,,,,,,,,,,,,,,,
207892,20221004.400000,2022-09-30,99.973333,99.973444,99.973389,M,I,0.000000,-1.389258e-07,0.000067,...,NaN,0,NaN,4,0,1,1,NaN,NaN,NaN
207774,20221006.400000,2022-09-30,99.960917,99.961083,99.961000,M,I,0.000000,-2.542594e-05,0.000065,...,NaN,0,NaN,4,0,1,1,NaN,NaN,NaN
207893,20221011.400000,2022-09-30,99.924986,99.925292,99.925139,M,I,0.000000,1.473347e-05,0.000068,...,NaN,0,NaN,4,0,1,1,NaN,NaN,NaN
207868,20221013.400000,2022-09-30,99.907556,99.907917,99.907736,M,I,0.000000,-8.270836e-05,0.000071,...,NaN,0,NaN,4,0,1,1,NaN,NaN,NaN
207430,20221015.201370,2022-09-30,99.921875,99.953125,99.937500,M,I,0.631148,3.735728e-05,0.000079,...,NaN,0,NaN,2,0,1,1,NaN,NaN,NaN


In [3]:
t_check = df['CALDT'].values[0]
if df['CALDT'].eq(t_check).all():
    t_current = t_check
else:
    warnings.warn('Quotes are from multiple dates.')
    t_current = None
f'These quotes are based on {pd.to_datetime(t_current):%Y-%m-%d}'

'These quotes are based on 2022-09-30'

In [4]:
treasury_metrics = df.copy()[['TDATDT','TMATDT','TCOUPRT','TDYLD','TDDURATN','TDPUBOUT']]
treasury_metrics.columns = ['issue date','maturity date','coupon rate','ytm','duration','outstanding']
treasury_metrics['ytm'] *= 365
treasury_metrics['duration'] /= 365
treasury_metrics['outstanding'] *= 1e6
treasury_metrics['ask-bid'] = df['TDASK'] - df['TDBID']
treasury_metrics['next cashflow date'] = (calc_cashflows(df) != 0).idxmax(1)

# Problem 1

## Part 1

For each issue, calculate its time-to-maturity, quoted in number of years, based on the maturity dates. Assume a year has exactly 365.25 days. 


In [5]:
treasury_metrics['Maturity'] = get_maturity_delta(treasury_metrics['maturity date'],t_current)
treasury_metrics['Maturity'].tail().to_frame('Maturity Dates')

,Maturity Dates
KYTREASNO,
207808,29.125257
207849,29.377139
207850,29.377139
207891,29.623546
207934,29.875428


## Part 2

- Calculate the dirty price for each bond issue as:

    **price = $\frac{1}{2}$ (TBID+TASK) + TDACCINT**

In [6]:
treasury_metrics['price'] = .5*(df['TDBID'] + df['TDASK']) + df['TDACCINT']
treasury_metrics['price'].to_frame("Dirty Price")

,Dirty Price
KYTREASNO,
207892,99.973389
207774,99.961000
207893,99.925139
207868,99.907736
207430,100.568648
...,...
207808,66.640625
207849,72.718750
207850,62.910156


## Yield-to-Maturity

- Calculate YTM for every bond issue. Sunday try to write a python function to calculate yield to maturity.

In [7]:
treasury_metrics

,issue date,maturity date,coupon rate,ytm,duration,outstanding,ask-bid,next cashflow date,Maturity,price
KYTREASNO,,,,,,,,,,
207892,2022-06-07,2022-10-04,0.000,0.024286,0.010959,NaN,0.000111,2022-10-04,0.010951,99.973389
207774,2021-10-07,2022-10-06,0.000,0.023730,0.016438,NaN,0.000167,2022-10-06,0.016427,99.961000
207893,2022-06-14,2022-10-11,0.000,0.024850,0.030137,NaN,0.000306,2022-10-11,0.030116,99.925139
207868,2022-04-14,2022-10-13,0.000,0.025917,0.035616,NaN,0.000361,2022-10-13,0.035592,99.907736
207430,2019-10-15,2022-10-15,1.375,0.028740,0.041096,2.323500e+10,0.031250,2022-10-15,0.041068,100.568648
...,...,...,...,...,...,...,...,...,...,...
207808,2021-11-15,2051-11-15,1.875,0.037857,20.506232,6.897200e+10,0.046875,2022-11-15,29.125257,66.640625
207849,2022-02-15,2052-02-15,2.250,0.037800,19.982853,6.300400e+10,0.062500,2023-02-15,29.377139,72.718750
207850,2022-02-15,2052-02-15,0.125,NaN,NaN,1.663300e+10,0.257812,2023-02-15,29.377139,62.910156
